In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [2]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [-9.9369061e-01,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        ...,
        [ 3.2816119e-02,  2.0456202e-02,  7.0173093e-03, ...,
          6.1736792e-02,  5.2754156e-02,  4.3626044e-02],
        [-3.4915574e-02, -4.1013200e-02, -4.6975732e-02, ...,
         -1.6504882e-02, -2.2655437e-02, -2.8794337e-02],
        [-4.9611825e-02, -5.1051393e-02, -5.2483197e-02, ...,
         -4.5252293e-02, -4.6711687e-02, -4.8165053e-02]],

       [[ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [ 1.0000000e-12,  1.0000000e-12,  1.0000000e-12, ...,
          1.0000000e-12,  1.0000000e-12,  1.0000000e-12],
        [ 1.0000000e-12, 

In [3]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [4]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4800,)


In [5]:
AO_ds.to_netcdf('GFDL-CM4-AOindex-NDJF-Daily-1980-2014.nc')

In [6]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(2.6828835, dtype=float32)
Coordinates:
    time     object 1994-12-05 12:00:00

In [7]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [8]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-4.0296197, dtype=float32)
Coordinates:
    time     object 1995-02-28 12:00:00

In [9]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [9]:
zLevData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/zg_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')

In [10]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [11]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4800, lat: 90, lon: 144)>
[62208000 values with dtype=float32]
Coordinates:
  * time     (time) object 1975-01-01 12:00:00 ... 2014-12-31 12:00:00
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    cell_methods:   time: mean
    cell_measures:  area: areacella
    interp_method:  conserve_order2
    original_name:  zg

In [12]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [13]:
#z145E40N

In [14]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [15]:
#z20E55N

In [16]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [17]:
#z75E55N

In [18]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [19]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [20]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [21]:
EUVal

<xarray.DataArray 'zg' (time: 4800)>
array([-0.01882858,  0.22620694,  0.41501371, ..., -0.07687441,
        0.1046467 , -0.10227694])
Coordinates:
  * time       (time) object 1975-01-01 12:00:00 ... 2014-12-31 12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 143.8

In [22]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [23]:
EU_ds.to_netcdf('GFDL-CM4-EUindex-NDJF-Daily-1980-2014.nc')